# Plot maximum water level increase and discharge

Using this script you can calculated the maximum increase in waterlevel
Per 1D node it calculates the maximum waterlevel and subtracts the initial waterlevel. In case of a dry bed the reference level of the calculation node is used as the initial waterlevel. The maximum water level increase is then plotted on an OpenStreetMap basemap. The same is done with the maximum discharge of flowlines

This script is meant as an example of what is possible in automating analysis of 3Di results. The user should install the requirements themselves using Conda or Pip on a Python 3 environment.

- Author: Emiel Verstegen, 2019

- email: emiel.verstegen@nelen-schuurmans.nl

In [ ]:
#import libraries, make sure you have them installed on your Python environment
from threedigrid.admin.gridresultadmin import GridH5ResultAdmin
from threedigrid.admin.gridadmin import GridH5Admin
import os
import numpy as np
import pandas as pd
import contextily as ctx
from geopandas import GeoDataFrame
from shapely.geometry import Point, LineString
from mpl_toolkits.basemap import Basemap
from pyproj import Proj, transform
import requests
from google_drive_downloader import GoogleDriveDownloader as gdd #Needs installing using pip, unlike the others

In [ ]:
#download testdata and unzip in testdata directory (https://drive.google.com/open?id=1xNhMZnWKxFP7on1reXxecq7XdTuJCSa2)


gdd.download_file_from_google_drive(file_id='1xNhMZnWKxFP7on1reXxecq7XdTuJCSa2',
                                    dest_path='./testdata/bergermeer.zip',
                                    unzip=True)

In [ ]:
#helper function to create OpenStreetMap background for plot
def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

In [ ]:
#Define location of raw results and gridadministration
result_path = 'testdata/'
nc = os.path.join(result_path,'results_3di.nc')
f = os.path.join(result_path,'gridadmin.h5')

#Load files into gr object
ga = GridH5Admin(f)
gr = GridH5ResultAdmin(f,nc)

In [ ]:
#subset only 1D nodes
nodes_1d = gr.nodes.subset('1D_ALL')

#Get the x,y-coordinates from the 1D nodes
nodes_1d_coordinates = nodes_1d.coordinates

#Read the whole timeserie for the waterlevel for the subset of 1D nodes into the wl variable
wl = nodes_1d.timeseries(start_time=0,end_time=gr.nodes.timestamps[-1]).s1

In [ ]:
#Read the reference (channel bottom level, stored in the z_coordinate) as a minimum waterlevel 
ref_lvl = gr.cells.subset('1D_ALL').z_coordinate

#Read the initial waterlevel (-9999 is a dry bed, so use maximum value of the reference level and the initial waterlevel)
wl_ini = np.amax([wl[0],ref_lvl],axis=0)

#Determine the maximum waterlevel, taking the greatest value of the maximum waterlevel and the reference level
wl_max = np.amax([np.amax(wl,axis=0),ref_lvl],axis=0)

#Determine the difference between the maximum waterlevel and the initial waterlevel
wl_max_increase = wl_max - wl_ini

In [ ]:
#Write data to dataframe
x = nodes_1d_coordinates[0]
y = nodes_1d_coordinates[1]
df = pd.DataFrame({'x': x, 'y': y, 'max_increase' : wl_max_increase}).sort_values(by=['max_increase'])

#output data to Comma Seperated Value file
#df.to_csv('testdata/output.csv')

In [ ]:
#Create an extra column containing Shapely points
df['geometry'] = df.apply(Point, axis=1)

#Create a GeoPandas dataframe in Webmercator projection (for projection on OpenStreetMap)
crs = {'init': 'epsg:{}'.format(ga.epsg_code)}
gdf = GeoDataFrame(df, crs=crs, geometry=df['geometry']).to_crs({'init': 'epsg:3857'})
gdf["category"] = pd.qcut(gdf['max_increase'],q=9,duplicates='drop', labels=[1,2,3,4,5,6,7,8,9])

In [ ]:
#Set up a Basemap object with the model extend (lower-left corner and upper-right corner)
inProj = Proj(init='epsg:{}'.format(ga.epsg_code))
outProj = Proj(init='epsg:4326')

llcrnr = transform(inProj,outProj,ga.get_model_extent()[0],ga.get_model_extent()[1])
urcrnr = transform(inProj,outProj,ga.get_model_extent()[2],ga.get_model_extent()[3])

m = Basemap(llcrnrlon=llcrnr[0],llcrnrlat=llcrnr[1],urcrnrlon=urcrnr[0],urcrnrlat=urcrnr[1],lat_ts=20,
            resolution='h',projection='merc',lon_0=-4.36,lat_0=54.5)

In [ ]:
#Plot the maximum waterlevel increase on the basemap with an OpenStreetMap background
ax = gdf.plot(column='category',figsize=(15, 15), alpha=0.8, markersize=12,cmap='Blues')
ax.set_axis_off()
ax.set_title("Maximum water level increase")
#add_basemap(ax, zoom=8,url=ctx.sources.OSM_A)

In [ ]:
#Make a subset of 1D flowlines
flowlines_1d = gr.lines.subset('1D')

#Request the discharge (q) from start to end of the simulation of alle 1D flowlines
q_1d = flowlines_1d.timeseries(start_time=0,end_time=gr.lines.timestamps[-1]).q

#request the coordinates of begin/end points of flowlines
flowlines_1d_coordinates = flowlines_1d.data['line_coords']

In [ ]:
#Calculate the maximum (absolute, so either direction) discharge in time of every line
q_max = np.amax(np.abs(q_1d),axis=0)[1:]

#Calculate the average discharge in type of every line
q_avg = np.mean(q_1d,axis=0)[1:]

In [ ]:
#Create a GeoPandas GeoDataFrame with LineStrings as geometry column
x_from = flowlines_1d_coordinates[0][1:]
y_from = flowlines_1d_coordinates[1][1:]
x_to = flowlines_1d_coordinates[2][1:]
y_to = flowlines_1d_coordinates[3][1:]
df_lines = pd.DataFrame({'x_from': x_from, 'y_from': y_from,'x_to': x_to, 'y_to': y_to, 'q_max' : q_max}).sort_values(by=['q_max'])
df_lines['point_from'] = df_lines[['x_from','y_from']].apply(Point, axis=1)
df_lines['point_to'] = df_lines[['x_to','y_to']].apply(Point, axis=1)


def points_to_line(a):
    return LineString([a[0],a[1]])

df_lines['geometry'] = df_lines[['point_from','point_to']].apply(points_to_line, axis=1)

gdf_lines = GeoDataFrame(df_lines, crs=crs, geometry=df_lines['geometry']).to_crs({'init': 'epsg:3857'})
gdf_lines["category"] = pd.qcut(gdf_lines['q_max'],q=9,duplicates='drop', labels=[1,2,3,4,5,6,7,8,9])

In [ ]:
#Set up a Basemap object with the model extend (lower-left corner and upper-right corner)
inProj = Proj(init='epsg:{}'.format(ga.epsg_code))
outProj = Proj(init='epsg:4326')

llcrnr = transform(inProj,outProj,ga.get_model_extent()[0],ga.get_model_extent()[1])
urcrnr = transform(inProj,outProj,ga.get_model_extent()[2],ga.get_model_extent()[3])

#m = Basemap(llcrnrlon=llcrnr[0],llcrnrlat=llcrnr[1],urcrnrlon=urcrnr[0],urcrnrlat=urcrnr[1],lat_ts=20,
#            resolution='h',projection='merc',lon_0=-4.36,lat_0=54.5)

In [ ]:
linewidth = gdf_lines.as_matrix(columns=gdf_lines.columns[4:5])
ax_lines = gdf_lines.plot(column='q_max',figsize=(15, 15), alpha=0.8, linewidth=gdf_lines['category'].as_matrix()/4,cmap='OrRd', scheme='quantiles')
ax_lines.set_axis_off()
ax_lines.set_title("Maximum discharge level increase")
#add_basemap(ax_lines, zoom=12,url=ctx.sources.OSM_A)